# `__getattr__` and `__getattribute__`

Properties and descriptors are used for managing specific attributes.

The `__getattr__` and `__getattribute__` operator overloading methods provide still other ways to intercept attribute fetches for class intances. Like properties and descriptors, they allow us to insert code to be run automatically when attributes are accessed. These two methods cal also be used in more general ways. Because they intercept arbitary names, they apply in broader roles such as delegation, but may also inur extra calls in some contexts, and are too dynamic to register in `dir` results.

Attribute fetch interception comes in two flavors, coded with two different methods:

- `__getattr__` is run for *undefined* attributes--because it is run only for attributes not stored on an instance or inherited from on of its classes, its use is straightforward.

- `__getattribute__` is run for *every* attribute--because it is all-inclusive, you must be cautious when using this method to avoid recursive loops by passing attribute accesses to a superclass.

These two methods are representatives of a set of attribute intercpetion methods that also includes `__setattr__` and `__delattr__`.

Unlike properties and descriptors, thse methods are part of Python's general *operator overloading* protocol--specially named methods of a class, inherited by subclasses, and run automatically when instances are used in the implied built-in operation. Like all normal methods of a class, they each recevie a first **self** argument when called, giving access to any required instance state information as well as other methods of the class in which they appear.

The `__getattr__` and `__getattribute__` methods are also more generic than properties and descriptors--they can be used to intercept access to any (or even all) instance attribute fetches, not just a single specific name. Because of this, these two methods are well suited to general *delegation*-based coding patterns--they can be used to implement wrapper (a.k.a. proxy) object that manage all attribute accesses for any embedded object. By contrast, we must define one property or descriptor for every  attribute we wich to intercept.

These two methods are more *narrowly focused* then the alternatives we considered earlier: they intercept attribute fetches only, not assignments. To also catch attribute changes by assignment, we must code a `__setattr__` methods--an operator overloading method run for every attribute fetch, which must take care to avoid recursive loops by routing attribute assignments through the instance namespace dicitionary os a superclass method. We can also code a `__delattr__` overloading method (which must avoid loop in the same way) to intercept attribute deletions. By constrast, properties and descriptors catch get, set and delete operations by design.

### The Basics

If a class defines or inherits the following methods, they will be run automatically when an instance is used in the context described by the commas to the right:

```pr 
def __getattr__(self, name):        # On undefined attribute fetch [obj.name]
def __getattribute__(self, name):   # On all attribute fetch [obj.name]
def __setattr__(self, name, value): # On all attribute assignment [obj.name=value]
def __delattr__(self, name):        # On all attribute deletion [del obj.name]
```

In all of these, **self** is the subject instance as usual, **name** is the string name of the attribute being accessed, and **value** is the object being assigned to the attribute. The two get methods normally return an attribute's value, and other two return nothing (None). All can raise exceptions to signal prohibited access.

For example, to catch every attribute fetch, we can use either of the first two previous methods, and to catch every attribute assignment we can use the third. The following uses `__getattr__` and workds on both Python 2.X and 3.X, not requiring new-style **object** in 2.X:

In [2]:
class Catcher:
    def __getattr__(self, name):
        print("Get: %s" % name)
    def __setattr__(self, name, value):
        print("Set: %s %s" % (name, value))
        
X = Catcher()
X.job               # Prints "Get: job"
X.pay               # Prints "Get: pay"
X.pay = 99          # Prints "Set: pay 99"

Get: job
Get: pay
Set: pay 99


Using `__getattribute__` workds exactly the same in this specific case, but requireds **object** derivation in 2.X (only), and has subtle looping potential, which we'll take up in the next section:

In [3]:
class Catcher(object):                      # Need (object) in 2.X only
    def __getattribute__(self, name):       # Works same as getattr here
        print("Get: %s" % name)             # But prone to loops on general
    def __setattr__(self, name, value):
        print("Set: %s %s" % (name, value))
        
X = Catcher()
X.job               # Prints "Get: job"
X.pay               # Prints "Get: pay"
X.pay = 99          # Prints "Set: pay 99"

Get: job
Get: pay
Set: pay 99


Such a coding structure can be used to implement the *delegation* design pattern. Because all attributes are routed to our interception methods generically, we can validate and pass them along to embedded, managed objects. The following class, for example, traces *every* attribute fetch made to another object passed to the wrapper (proxy) class:

In [4]:
class Wrapper:
    def __init__(self, object):
        self.wrapped = object                   # Save object
    def __getattr__(self, attrname):
        print("Trace: " + attrname)             # Trace fetch
        return getattr(self.wrapped, attrname)  # Delegate fetch

X = Wrapper([1, 2, 3])
X.append(4)                     # Prints "Trace: append"
print(X.wrapped)                # Prints "[1, 2, 3, 4]"
    

Trace: append
[1, 2, 3, 4]


There is no such analog for properties and descriptors, short of coding accessors for *every* possible attribute wrapped object. On the other hand, when such generality is not required, generic accessor methods may incur additional calls for assignments in some contexts.

### Avoiding loops in attribute interception methods

These methods are generally more straightforward to use; their only substantially compex aspect is the potential for *looping* (a.k.a. recursing). Because `__getattr__` is called for undefined attributes only, it can freely fetch other attributes within its own code. However, because `__getattribute__` and `__setattr__` are run for *all* attributes, their code needs to be careful when accessing other attributes to avoid calling themselves again and triggering a recursive loop.

For example, another attribute fetch run inside a `__getattribute__` method's code will trigger `__getattribute__` again, and the code will sually loop until memory is exhausted:

In [6]:
def __get__(self, name):
    x = self.other                  # LOOPS!

Techincally, this methods is ever more loop-prone than this may imply--a **self** atribute reference run *anywhere* in a class that defines this method will trigger `__getattribute__`, and also has the potential to loop depending on the class's logic. this is normally desired behavior--intercepting every attribute fetch is thsi method's purpose, after all--but you should be aware that this method catches all attribute fetches wherever they are coded. When coded within `__getattribute__` itself, thsi almost always causes a loop. To avoid this loop, route the fetch through a higher superclass instead to skip this level's version--because the **object** class is always a new -styple superclass, it serves well in this role:

In [7]:
def __getattribute__(self, name):
    x = object.__getattribute__(self, 'other')  # Force higher to avoid me

For **__setattr__**, the situation, assignment *any* attirbute inside this method triggers `__setattr__` again and may create a similar loop:

In [8]:
def __setattr__(self, name, value):
    self.other = value                  # Recurs (and might LOOP!)

Here too, **self** attribute assignments *anywhere* in a class defining this method trigger `__setattr__` as well, though the potential for looping is much stronger when they show up in `__setattr__` itself. To work around this problem, you can assign the attribute as a key in the instance's `__dict__` namespace dictionary instead. This avoids direct attribute assignment:

In [ ]:
def __setattr__(self, name, value):
    self.__dict__['other'] = value      # Use attr dict to avoid me

Although it's a less traditional approach, `__setattr__` can also pass its own attribute assignment to a higher superclass to avoid looping, just like `__getattribute__` (and per the upcoming note, this scheme is sometimes preferred):

In [10]:
def __setattr__(self, name, value):
    object.__setattr__(self, 'other', value)        # Force higher to avoid me.

By contrast, though, we *cannot* use the `__dict__` trick to avoid loops in `__getattribute__`:

In [ ]:
def __getattribute__(self, name):
    x = self.__dict__['other']              # Loops!

Fetching the `__dit__` attribute itself triggers `__getattribute__` again, causing a recursive loop. Strnage but true!